In [ ]:
# -*- coding: utf-8 -*-
# -*- author : Vincent Roduit -*-
# -*- date : 2023-11-25 -*-
# -*- Last revision: 2023-11-25 -*-
# -*- python version : 3.11.6 -*-
# -*- Description: Notebook that summarize results-*-

# <center> CS -433 Machine Learning </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Road Segmentation </center>
---

### Preparing environment for Google Colaboratory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/ml-project-2-team-slo/source

### Imports

In [23]:
#import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from constants import *

#import model parameters
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from torchvision import models

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
#import files
from data_processing import*
from visualization import visualize, visualize_patch
import constants
from test_data import TestData

#import models
from cnn import Basic_CNN, Advanced_CNN
from logistic_regression import LogisticRegression

In [4]:
# Set random seed for reproducibility
torch.manual_seed(0)

## 1. Data wrangling and visualization

In [45]:
myDatas = AdvancedProcessing(standardize=False, num_samples=500)
myDatas.proceed()

Loading data...
Done!
Splitting data...
Done!
Creating patches...
Creating patches for training set...
Rotation for 45 degrees
Rotation for 135 degrees
Rotation for 225 degrees
Rotation for 315 degrees
Batch 1/15
Batch 4/15
Batch 7/15
Batch 10/15
Batch 13/15
end process...
Creating patches for validation set...
Done!
Creating dataloader...
Done!


In [31]:
myTestDatas = TestData(standardize=True)
myTestDatas.proceed()

Loading data...
Done!
Standardizing...
Done!
Formatting data...
Done!
Creating dataloader...
Done!


## 2. Define and train models

### 2.1 Logistic regression

A first attempt could be to try with some linear model. The first approach here is to use a simple logistic regression. In order to use a logistic regression, one need to extract feature from the image. A choice could be to use the mean and the standard deviation as features. The following section will present these approach.

In [6]:
LogisticData = BasicProcessing()
LogisticData.load_data()
LogisticData.create_patches()

[autoreload of cnn failed: Traceback (most recent call last):
  File "C:\Users\ylaar\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\ylaar\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "c:\Users\ylaar\anaconda3\Lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\ylaar\switchdrive\epfl\ml\ml-project-2-team-slo\source\cnn.py", line 107, in <module>
    class Basic_CNN(CNN):
  File "c:\Users\ylaar\switchdrive\epfl\ml\ml-project-2-team-slo\source\cnn.py", line 110, in Basic_CNN
    patch_size=constan

Loading data...
Done!
Creating patches...
Done!


In [ ]:
LogReg = LogisticRegression(LogisticData.imgs_patches, LogisticData.gt_imgs_patches)
LogReg.compute_vectors()

In [ ]:
plt.scatter(LogReg.X[:, 0], LogReg.X[:, 1], c=LogReg.Y, edgecolors="k", cmap=plt.cm.Paired)
plt.show()

A problem already arises. The datas are not linearly separable. Let's still try to train the model.

In [ ]:
LogReg.train()
LogReg.predict()

In [ ]:
print(f'From this model, the accuracy is {LogReg.accuracy*100:.2f}% and the F1 score is {LogReg.f1*100:.2f}%')

The unsatisfactory results tend us to move to Convolutional Networks, which are more suitable for image datas.

### 2.2 Basic Convolutional Neural Network

In [17]:
# Define the loss function
criterion = nn.BCEWithLogitsLoss()

#### Training on different patch size

Basic model

In [41]:
# Define the patch size
patch_size = 16

# Load data
myDatas = AdvancedProcessing(standardize=False, aug_patch_size=patch_size)
myDatas.proceed()

# Define the model
cnn = Basic_CNN(patch_size)

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    criterion,
    myDatas.train_dataloader,
    myDatas.validate_dataloader,
    num_epochs=20)

torch_file_name = f'basic_cnn_{patch_size}.pth'
# Save the model
torch.save(cnn, os.path.join(MODELS_DIR, torch_file_name))

Epoch 1/20, Loss: 0.668280284690857, Validation Accuracy: 0.7355, F1 score: nan
Epoch 2/20, Loss: 0.6818244309425354, Validation Accuracy: 0.7069, F1 score: nan
Epoch 3/20, Loss: 0.6145328700637818, Validation Accuracy: 0.7355, F1 score: nan
Epoch 4/20, Loss: 0.702554199180603, Validation Accuracy: 0.6058, F1 score: 0.4335
Epoch 5/20, Loss: 0.6768155156326294, Validation Accuracy: 0.6287, F1 score: 0.4181
Epoch 6/20, Loss: 0.6486707897186279, Validation Accuracy: 0.6654, F1 score: 0.4607
Epoch 00006: reducing learning rate of group 0 to 1.0000e-04.
Epoch 7/20, Loss: 0.6627873078536988, Validation Accuracy: 0.6530, F1 score: 0.4768
Epoch 8/20, Loss: 0.6494840443038941, Validation Accuracy: 0.6754, F1 score: 0.4821
Epoch 9/20, Loss: 0.6358836765480042, Validation Accuracy: 0.6989, F1 score: 0.4778
Epoch 00009: reducing learning rate of group 0 to 1.0000e-05.
Epoch 10/20, Loss: 0.6388512574768066, Validation Accuracy: 0.6943, F1 score: 0.4833
Epoch 11/20, Loss: 0.6409749675178528, Validat

In [ ]:
patch_size = 32

# Load data
myDatas = AdvancedProcessing(standardize=False, aug_patch_size=patch_size)
myDatas.proceed()

# Define the model
cnn = Basic_CNN(patch_size)

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    criterion,
    myDatas.train_dataloader,
    myDatas.validate_dataloader,
    num_epochs=20)

torch_file_name = f'basic_cnn_{patch_size}.pth'
# Save the model
torch.save(cnn, os.path.join(MODELS_DIR, torch_file_name))

In [ ]:
patch_size = 64

# Load data
myDatas = AdvancedProcessing(standardize=False, aug_patch_size=patch_size)
myDatas.proceed()

# Define the model
cnn = Basic_CNN(patch_size)

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    criterion,
    myDatas.train_dataloader,
    myDatas.validate_dataloader,
    num_epochs=20)

torch_file_name = f'basic_cnn_{patch_size}.pth'
# Save the model
torch.save(cnn, os.path.join(MODELS_DIR, torch_file_name))

In [ ]:
patch_size = 128

# Load data
myDatas = AdvancedProcessing(standardize=False, aug_patch_size=patch_size)
myDatas.proceed()

# Define the model
cnn = Basic_CNN(patch_size)

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    criterion,
    myDatas.train_dataloader,
    myDatas.validate_dataloader,
    num_epochs=20)

torch_file_name = f'basic_cnn_{patch_size}.pth'
# Save the model
torch.save(cnn, os.path.join(MODELS_DIR, torch_file_name))

Advanced CNN

In [ ]:
patch_size = 64

# Load data
myDatas = AdvancedProcessing(standardize=False, aug_patch_size=patch_size)
myDatas.proceed()

# Define the model
cnn = Basic_CNN(patch_size)

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    criterion,
    myDatas.train_dataloader,
    myDatas.validate_dataloader,
    num_epochs=20)

torch_file_name = f'advanced_cnn_{patch_size}.pth'
# Save the model
torch.save(cnn, os.path.join(MODELS_DIR, torch_file_name))

In [ ]:
patch_size = 128

# Load data
myDatas = AdvancedProcessing(standardize=False, aug_patch_size=patch_size)
myDatas.proceed()

# Define the model
cnn = Advanced_CNN(patch_size)

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    criterion,
    myDatas.train_dataloader,
    myDatas.validate_dataloader,
    num_epochs=20)

torch_file_name = f'advanced_cnn_{patch_size}.pth'
# Save the model
torch.save(cnn, os.path.join(MODELS_DIR, torch_file_name))

#### Color standarization

In [ ]:
patch_size = 128

# Load data
myDatas = AdvancedProcessing(standardize=True, aug_patch_size=patch_size)
myDatas.proceed()

# Define the model
cnn = Advanced_CNN(patch_size)

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    criterion,
    myDatas.train_dataloader,
    myDatas.validate_dataloader,
    num_epochs=20)

torch_file_name = f'advanced_cnn_color_{patch_size}.pth'
# Save the model
torch.save(cnn, os.path.join(MODELS_DIR, torch_file_name))

#### Different optimizer

AdamW

In [ ]:
patch_size = 128

# Load data
myDatas = AdvancedProcessing(standardize=False, aug_patch_size=patch_size)
myDatas.proceed()

# Define the model
cnn = Advanced_CNN(patch_size)

# Define the optimizer
optimizer = torch.optim.AdamW(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    criterion,
    myDatas.train_dataloader,
    myDatas.validate_dataloader,
    num_epochs=20)

torch_file_name = f'advanced_cnn_{patch_size}_adamw.pth'
# Save the model
torch.save(cnn, os.path.join(MODELS_DIR, torch_file_name))

SGD with Nesterov Momentum

In [ ]:
patch_size = 128

# Load data
myDatas = AdvancedProcessing(standardize=False, aug_patch_size=patch_size)
myDatas.proceed()

# Define the model
cnn = Advanced_CNN(patch_size)

# Define the optimizer
optimizer = torch.optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9, nesterov=True)
# Define the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    criterion,
    myDatas.train_dataloader,
    myDatas.validate_dataloader,
    num_epochs=20)

torch_file_name = f'advanced_cnn_{patch_size}_nesterov.pth'
# Save the model
torch.save(cnn, os.path.join(MODELS_DIR, torch_file_name))